In [1]:
def cyclotomic_class(n, q, s):
    # calculate the q-cyclotomic class of s in (Z/nZ)*
    if n.gcd(q)>1:
        return False
    # since X^n-1 is separable over F_q
    C=[s]
    t=s*q %n
    while not (t==s):
        C=C+[t]
        t=t*q %n
    return C
        
    
    

In [2]:
cyclotomic_class(23,2,1)

[1, 2, 4, 8, 16, 9, 18, 13, 3, 6, 12]

In [3]:
def all_cyclotomic_class(n, q):
    if n.gcd(q)>1:
        return False
    PC=C=[0]    # put all precedent values in one list
    AC=[C]      # list of all classes of s
    for s in [1..n-1]:
        if not(s in PC):
            C= cyclotomic_class(n, q, s)
            PC=PC+C
            AC=AC+[C]
    return AC

In [4]:
all_cyclotomic_class(23, 2)

[[0],
 [1, 2, 4, 8, 16, 9, 18, 13, 3, 6, 12],
 [5, 10, 20, 17, 11, 22, 21, 19, 15, 7, 14]]

In [5]:
def order_n_element(n, q):
    if n.gcd(q)>1:
        return False
    m=IntegerModRing(n)(q).multiplicative_order()
    F.<a>=GF(q^m)          # normally a is exactly the generator of the cyclotomic extension, with order 2^m not n
    return(a^((q^m-1) //n))  # return the power of alpha in the cyclic group
    

# Here 'a' only represents a root of the defined irre, other irre is primitive, then 'a' is also primitive, an example:

In [2]:
FF.<a0>=GF(2^8, modulus=x^8+x^4+x^3+x+1); FF

Finite Field in a0 of size 2^8

In [4]:
a0.multiplicative_order()    # 51 != 2^8-1

51

In [6]:
F.<a>=GF(2^11); a.multiplicative_order()

2047

In [7]:
order_n_element(23,2)

a^8 + a^6 + a

In [8]:
def cyclotomic_poly (n, q, alpha, s, x):
    # find the nth cyclotomic polynomial of alpha^s over F_q, or its minimum polynomial
    F=alpha.parent()  # the nth cyclotomic extension
    C=cyclotomic_class(n,q, s)   
    d= len(C)    # the degree of the polynomial or the number of orbit alpha^s
    V=F.vector_space()    # expliciting the vector space of F using the basis: 1, alpha, alpha^2 ..
    # c_0+c_1 alpha^s+ c_2 alpha^s*s +.. =0, thus all solution is (coefficients of) minimal poly considering the degree
    # more generally, we can relax a bit the degree constraint
    L=[V(alpha^(s*i)) for i in [0 .. d]]
    K=matrix(L).left_kernel().basis()
    R.<x>=PolynomialRing(GF(q))   # construct a polynomial ring
    P=[ K[0][i]/K[0][d]*x^i for i in [0 .. d] ]
    return (sum(P))
    

In [9]:
alpha=order_n_element(23,2)
cyclotomic_poly(23, 2, alpha, 1, x), order_n_element(23,2).minimal_polynomial()    # directly use the mini-poly function in SageMath

/Applications/SageMath/local/lib/python3.7/site-packages/sage/repl/ipython_kernel/__main__.py:6: DeprecationWarning: The default value for map will be changing to True.  To keep the current behavior, explicitly pass map=False.
See https://trac.sagemath.org/28481 for details.


(x^11 + x^9 + x^7 + x^6 + x^5 + x + 1, x^11 + x^9 + x^7 + x^6 + x^5 + x + 1)

In [10]:
def fact_x_minus_1(n,q,x):
    # factorization of x^n-1 over F_q, here we should make x as python variable for coherent transmission 
    if n.gcd(q)>1:
        return False
    alpha=order_n_element(n,q)
    AC=all_cyclotomic_class(n,q)
    return ([cyclotomic_poly(n,q, alpha, C[0], x) for C in AC])

In [11]:
f=fact_x_minus_1(23,2,x)
f, mul(u for u in f)

/Applications/SageMath/local/lib/python3.7/site-packages/sage/repl/ipython_kernel/__main__.py:6: DeprecationWarning: The default value for map will be changing to True.  To keep the current behavior, explicitly pass map=False.
See https://trac.sagemath.org/28481 for details.


([x + 1,
  x^11 + x^9 + x^7 + x^6 + x^5 + x + 1,
  x^11 + x^10 + x^6 + x^5 + x^4 + x^2 + 1],
 x^23 + 1)

In [12]:
def BCH_generator(n, q, alpha, t, x):
    # generator polynomial of a BCH code strict, over F_q, length n with distance prescribed 2*t+1
    P=lcm([cyclotomic_poly(n,q,alpha,i,x) for i in [1 .. 2*t]])
    return P

In [13]:
R.<x>=PolynomialRing(GF(2))
F.<alpha>=GF(2^4, modulus=x^4+x^3+1)
alpha.multiplicative_order()
BCH_generator(15,2,alpha,2,x)

/Applications/SageMath/local/lib/python3.7/site-packages/sage/repl/ipython_kernel/__main__.py:6: DeprecationWarning: The default value for map will be changing to True.  To keep the current behavior, explicitly pass map=False.
See https://trac.sagemath.org/28481 for details.


x^8 + x^4 + x^2 + x + 1

In [14]:
def Extended_Euclid_bounded(a,b,t):
    # find r_{n+1}=ua+vb where deg(r_n+1)<t and deg(r_n)>=t
    if a.degree()>b.degree(): 
        r0=a; r1=b
    else:
        r0=b; r1=a
    U0=1; U1=0
    V0=0; V1=1
    r,q,U,V=0,0,0,0
    while (r0.degree()>t or r0.degree()==t):
        q=r0//r1; r=r0%r1
        r0=r1; r1=r
        U=U0+q*U1; V=V0+q*V1
        U0=U1; V0=V1
        U1=U; V1=V   # r1=U1*a+V1*b
    return (r0, U0, V0) # r0=U0*a+V0*b
        

In [15]:
# calculate TD4 exo 4
F.<a>=GF(16, modulus=x^4+x^3+1)
A.<x>=PolynomialRing(F)
P=x^4
Q=a^3+a+1+(a^3+a)*x+(a^2+a)*x^2+(a^3+a+1)*x^3
Extended_Euclid_bounded(P,Q,2)  

(a^3 + a^2, a*x + a^3 + a^2, (a^3 + a^2 + 1)*x^2 + (a^3 + a^2)*x + a^2 + 1)

In [16]:
v=Extended_Euclid_bounded(P,Q,2)[2]
v.subs(x=0)

a^2 + 1

In [17]:
def De_BCH(F, n, alpha, t, x):
    # Since our field is F_2, the return of sigma function will suffice to find the error e
    C=[F.subs(x=alpha)]
    for i in [2 .. 2*t]:
        C=C+[F.subs(x=alpha^i)]
    S=C[0]
    for i in [1 .. 2*t-1]:
        S=S+C[i]*x^i
    v=Extended_Euclid_bounded(x^(2*t),S,t)[2]
    r=Extended_Euclid_bounded(x^(2*t),S,t)[0]
    v0=v.subs(x=0)
    sigma=v/v0
    w=r/v0
    X=[]
    for i in [0 .. n-1]:
        if sigma.subs(x=(1/alpha^i))==0:
            X=X+[[i, alpha^i]]  
    return X     # X[n]=[i, alpha^i], e=sum(alpha^i)
    
        
    
    
    
    

In [18]:
# Calculate  example 27 in the course notes:
R.<x>=PolynomialRing(GF(2))
F.<a>=GF(2^4, modulus=x^4+x+1)
a.multiplicative_order()

15

In [19]:
F=x^5+x^4+x^3+1
De_BCH(F, 16, a, 2, x)  # thus the errors are in the position 8e and 14e, i.e e(x)=x^8+x^14

[[8, a^2 + 1], [14, a^3 + 1]]

In [21]:
C = codes.CyclicCode(generator_pol = x^11 + x^9 + x^7 + x^6 + x^5 + x + 1, length = 23); C

[23, 12] Cyclic Code over GF(2)

In [23]:
MC=C.generator_matrix(); MC   # its generator matrix

[1 1 0 0 0 1 1 1 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0]
[0 1 1 0 0 0 1 1 1 0 1 0 1 0 0 0 0 0 0 0 0 0 0]
[0 0 1 1 0 0 0 1 1 1 0 1 0 1 0 0 0 0 0 0 0 0 0]
[0 0 0 1 1 0 0 0 1 1 1 0 1 0 1 0 0 0 0 0 0 0 0]
[0 0 0 0 1 1 0 0 0 1 1 1 0 1 0 1 0 0 0 0 0 0 0]
[0 0 0 0 0 1 1 0 0 0 1 1 1 0 1 0 1 0 0 0 0 0 0]
[0 0 0 0 0 0 1 1 0 0 0 1 1 1 0 1 0 1 0 0 0 0 0]
[0 0 0 0 0 0 0 1 1 0 0 0 1 1 1 0 1 0 1 0 0 0 0]
[0 0 0 0 0 0 0 0 1 1 0 0 0 1 1 1 0 1 0 1 0 0 0]
[0 0 0 0 0 0 0 0 0 1 1 0 0 0 1 1 1 0 1 0 1 0 0]
[0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 1 1 1 0 1 0 1 0]
[0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 1 1 1 0 1 0 1]

In [24]:
MP=C.parity_check_matrix(); MP  # its parity-check matrix

[1 0 1 0 0 1 0 0 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0]
[0 1 0 1 0 0 1 0 0 1 1 1 1 1 0 0 0 0 0 0 0 0 0]
[0 0 1 0 1 0 0 1 0 0 1 1 1 1 1 0 0 0 0 0 0 0 0]
[0 0 0 1 0 1 0 0 1 0 0 1 1 1 1 1 0 0 0 0 0 0 0]
[0 0 0 0 1 0 1 0 0 1 0 0 1 1 1 1 1 0 0 0 0 0 0]
[0 0 0 0 0 1 0 1 0 0 1 0 0 1 1 1 1 1 0 0 0 0 0]
[0 0 0 0 0 0 1 0 1 0 0 1 0 0 1 1 1 1 1 0 0 0 0]
[0 0 0 0 0 0 0 1 0 1 0 0 1 0 0 1 1 1 1 1 0 0 0]
[0 0 0 0 0 0 0 0 1 0 1 0 0 1 0 0 1 1 1 1 1 0 0]
[0 0 0 0 0 0 0 0 0 1 0 1 0 0 1 0 0 1 1 1 1 1 0]
[0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 1 0 0 1 1 1 1 1]

In [46]:
I1=matrix(GF(2), 1, 23, [1 for i in [1 ..23]]); I2=matrix(GF(2), 1, 1, [1])
block_matrix(1, 2, [I1, I2])

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1|1]

In [77]:
Z0=matrix([0 for i in [1 .. 11]]); Z0

[0 0 0 0 0 0 0 0 0 0 0]

In [71]:
Me=block_matrix(2,2,[ [MP, Z0.transpose()], [I1, I2] ]); Me

[1 0 1 0 0 1 0 0 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0|0]
[0 1 0 1 0 0 1 0 0 1 1 1 1 1 0 0 0 0 0 0 0 0 0|0]
[0 0 1 0 1 0 0 1 0 0 1 1 1 1 1 0 0 0 0 0 0 0 0|0]
[0 0 0 1 0 1 0 0 1 0 0 1 1 1 1 1 0 0 0 0 0 0 0|0]
[0 0 0 0 1 0 1 0 0 1 0 0 1 1 1 1 1 0 0 0 0 0 0|0]
[0 0 0 0 0 1 0 1 0 0 1 0 0 1 1 1 1 1 0 0 0 0 0|0]
[0 0 0 0 0 0 1 0 1 0 0 1 0 0 1 1 1 1 1 0 0 0 0|0]
[0 0 0 0 0 0 0 1 0 1 0 0 1 0 0 1 1 1 1 1 0 0 0|0]
[0 0 0 0 0 0 0 0 1 0 1 0 0 1 0 0 1 1 1 1 1 0 0|0]
[0 0 0 0 0 0 0 0 0 1 0 1 0 0 1 0 0 1 1 1 1 1 0|0]
[0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 1 0 0 1 1 1 1 1|0]
[---------------------------------------------+-]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1|1]

In [74]:
D=codes.LinearCode(Me).dual_code(); D

[24, 12] linear code over GF(2)

In [75]:
D.is_self_dual()

True